In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from service.Classifier.DataLoader.DataLoader import DataLoader
from service.Classifier.Enums.priority import Priority
from service.Classifier.DataLoader.P2DataLoader import P2DataLoader
from service.Classifier.DataLoader.P3DataLoader import P3DataLoader
from service.Classifier.DataLoader.P4DataLoader import P4DataLoader
from service.Classifier.Model.XGBRegressionModel import XGBRegressionModel
from service.Classifier.PreProcessing.RegressionModelPreProcessor import RegressionModelPreProcessor
from service.Classifier.DataSplit.TrainTestSplit import TrainTestSplit
from service.Classifier.Enums.dataTypeEnum import DataType

/Users/fjirigesi/Documents/OOSLAClassifier/service/Classifier/Model/XGBRegressionModel.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  from pandas import np


In [2]:
def initializeByPriority(priority):
    # load new updated data via Features class
    if priority == Priority.P2.value:
        return P2DataLoader()
    elif priority == Priority.P3.value:
        return P3DataLoader()
    elif priority == Priority.P4.value:
        return P4DataLoader()
    return None

In [3]:
priority = Priority.P2.value
def extract_data(priority):
    dataLoader = initializeByPriority(priority)
    data_df = dataLoader.loadTrainingRawData()
    # split data into train test 4:1
    dataSplit = TrainTestSplit()
    train_df, test_df = dataSplit.split(data_df)

    # generate features
    train_df, feature_names = dataLoader.transformRawDataToFeatures(train_df, DataType.TRAINDATA.value)
    test_df, feature_names = dataLoader.transformRawDataToFeatures(test_df, DataType.VALIDATION.value)
    # pre-processing the data based on model type
    preprocessor = RegressionModelPreProcessor(feature_names)
    train_X, train_y, test_X, test_y = preprocessor.preprocessing(train_df, test_df)
    
    return (train_X, train_y, test_X, test_y, train_df, test_df)

In [4]:
import xgboost as xgb
import numpy as np
import pandas as pd
from service.Classifier.Enums.LabelEnum import LabelEnum
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from service.Classifier.PreProcessing.Utils.dataValidator import DataValidator

def evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df):

    model = xgb.XGBRegressor(max_depth=5, n_estimators=100, learning_rate=0.05).fit(train_X, train_y)
    predictions = np.rint(model.predict(test_X)).astype(np.int64)
    submission = pd.DataFrame({'ID': test_df['keyID'],
                               LabelEnum.CLOSEDDAY.value: predictions, 'Actual': test_y})
    rmse = np.sqrt(mean_squared_error(predictions, test_df[LabelEnum.CLOSEDDAY.value]))
    print("RMSE: %f" % rmse)
    mae = mean_absolute_error(predictions, test_y)
    print("MAE: %f" % mae)
    DataValidator().valid(submission, priority, test_df)
    print(confusion_matrix(submission["truth"], submission["prediction"]))
    print(classification_report(submission["truth"], submission["prediction"]))
    
    return (test_y, predictions, submission["truth"], submission["prediction"])

In [5]:
priority = Priority.P2.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p2_regression_true, p2_regression_pred, p2_classification_true, p2_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: NATBEMCR
The project is not in embedding: PX
The project is not in embedding: GGRIP
The project is not in embedding: PROBLEM
The project is not in embedding: CSCNPLAT
RMSE: 7.979045
MAE: 5.378582
[[263 125]
 [ 40 235]]
              precision    recall  f1-score   support

       False       0.87      0.68      0.76       388
        True       0.65      0.85      0.74       275

    accuracy                           0.75       663
   macro avg       0.76      0.77      0.75       663
weighted avg       0.78      0.75      0.75       663



In [6]:
train_df

,keyID,priority,chainLength,longTimeHold,lateTriageDays,InitialActivityCount,InitialActionCount,recentActionCount,recentActivityCount,projectCoverage,ResolvedDay,ClosedDay
0,BILLING-16571,P2,0,0,0,3.2,0.0,0.2,2.8,0.038925,13,15
1,CANCEL-6059,P2,0,0,0,3.6,1.0,0.2,3.0,0.004836,12,14
2,REGSTRN-12121,P2,0,0,0,1.8,0.6,0.4,0.8,0.150463,22,26
3,BILLING-16536,P2,1,0,0,2.4,0.2,0.2,1.8,0.009579,19,19
4,BILLING-16547,P2,3,0,0,4.8,0.4,0.8,2.6,0.004129,14,16
...,...,...,...,...,...,...,...,...,...,...,...,...
2626,COMS-7986,P2,0,0,0,0.8,0.0,0.2,1.2,0.008488,12,12
2627,ONBAML-13038,P2,2,1,0,4.4,0.4,0.4,2.4,0.049914,11,11
2628,BILLING-16593,P2,1,1,0,2.4,0.4,0.4,1.6,0.035085,13,13
2629,USERCRUD-10078,P2,3,2,0,2.4,0.2,0.2,2.4,0.008073,33,34


In [7]:
priority = Priority.P3.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p3_regression_true, p3_regression_pred, p3_classification_true, p3_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: RISKVRTB
The project is not in embedding: MPPMO
The project is not in embedding: NRP
The project is not in embedding: WARROOM
The project is not in embedding: WEBRES
The project is not in embedding: PROBLEM
RMSE: 16.354038
MAE: 10.700118
[[467 152]
 [ 47 181]]
              precision    recall  f1-score   support

       False       0.91      0.75      0.82       619
        True       0.54      0.79      0.65       228

    accuracy                           0.77       847
   macro avg       0.73      0.77      0.73       847
weighted avg       0.81      0.77      0.78       847



In [11]:
priority = Priority.P4.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p4_regression_true, p4_regression_pred, p4_classification_true, p4_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: SDOCATOP
The project is not in embedding: VATSEC
RMSE: 21.656161
MAE: 13.426124
[[241  96]
 [ 38  92]]
              precision    recall  f1-score   support

       False       0.86      0.72      0.78       337
        True       0.49      0.71      0.58       130

    accuracy                           0.71       467
   macro avg       0.68      0.71      0.68       467
weighted avg       0.76      0.71      0.73       467



In [9]:
# # regression results
# regression_true = pd.concat([p2_regression_true, p3_regression_true])
# regression_true = pd.concat([regression_true, p4_regression_true])

# regression_pred = np.concatenate((p2_regression_pred, p3_regression_pred), axis=0)
# regression_pred = np.concatenate((regression_pred, p4_regression_pred), axis=0)

# ## classification results
# classification_true = pd.concat([p2_classification_true, p3_classification_true])
# classification_true = pd.concat([classification_true, p4_classification_true])

# classification_pred = pd.concat([p2_classification_pred, p3_classification_pred])
# classification_pred = pd.concat([classification_pred, p4_classification_pred])

In [10]:
# print("---Dynamic Features (including Activity Sequ) results on all data 75%---")
# rmse = np.sqrt(mean_squared_error(regression_pred, regression_true))
# print("RMSE: %f" % rmse)
# mae = mean_absolute_error(regression_pred, regression_true)
# print("MAE: %f" % mae)
# print(confusion_matrix(classification_true, classification_pred))
# print(classification_report(classification_true, classification_pred))